In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image

plt.style.use('ggplot')

In [ ]:
annot = pd.read_parquet(r"D:\Notebook\1705\Dataset\annot.parquet")
imgs = pd.read_parquet(r"D:\Notebook\1705\Dataset\img.parquet")
img_fns = glob(r"D:\Notebook\1705\Dataset\train_val_images\train_images\*")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(plt.imread(img_fns[0]))
ax.axis('off')
plt.show()

In [ ]:
image_id = img_fns[0].split("\\")[-1].split('.')[0]
annot.query('image_id == @image_id')


In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20, 20))
axs = axs.flatten()
for i in range(25):
    axs[i].imshow(plt.imread(img_fns[i]))
    axs[i].axis('off')
    image_id = img_fns[i].split('\\')[-1].rstrip('.jpg')
    n_annot = len(annot.query('image_id == @image_id'))
    axs[i].set_title(f'{image_id} - {n_annot}')
plt.show()

In [ ]:
pip install pytesseract

In [ ]:
import pytesseract

# Set Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Example call
print(pytesseract.image_to_string(img_fns[11], lang='eng'))


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(plt.imread(img_fns[11]))
ax.axis('off')
plt.show()

In [ ]:
pip install easyocr


In [ ]:
import easyocr

reader = easyocr.Reader(['en'], gpu = True)

In [ ]:
results = reader.readtext(img_fns[11])


In [ ]:
pd.DataFrame(results, columns=['bbox','text','conf'])


In [ ]:
pip install keras-ocr==0.9.3


In [ ]:
import easyocr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Function to plot and compare EasyOCR results
def plot_compare(img_fn, easyocr_df):
    img_id = img_fn.split('/')[-1].split('.')[0]
    fig, axs = plt.subplots(1, 1, figsize=(15, 10))

    easy_results = easyocr_df.query('img_id == @img_id')[['text', 'bbox']].values.tolist()
    easy_results = [(x[0], np.array(x[1])) for x in easy_results]

    image = plt.imread(img_fn)
    for text, bbox in easy_results:
        bbox = bbox.reshape(-1, 2)
        # Draw bounding box
        axs.plot(np.r_[bbox[:, 0], bbox[0, 0]], np.r_[bbox[:, 1], bbox[0, 1]], 'g-', linewidth=2)
        # Display text
        axs.text(bbox[0, 0], bbox[0, 1], text, va='top', fontsize=12, color='k')

    axs.imshow(image)
    axs.set_title('EasyOCR results', fontsize=24)
    plt.show()

# List to store EasyOCR results for all images
dfs = []

# Loop over images to extract text using EasyOCR
for img_fn in tqdm(img_fns[:25]):
    result = reader.readtext(img_fn)
    img_id = img_fn.split('\\')[-1].split('.')[0]
    img_df = pd.DataFrame(result, columns=['bbox', 'text', 'conf'])
    img_df['img_id'] = img_id
    dfs.append(img_df)

# Concatenate results into a single DataFrame
easyocr_df = pd.concat(dfs)

# Loop over images and plot EasyOCR results
for img_fn in img_fns[:25]:
    plot_compare(img_fn, easyocr_df)


In [ ]:
import easyocr
import pandas as pd
from tqdm import tqdm

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# List to store extracted text from images
extracted_text = []

# Loop over images to extract text using EasyOCR
for img_fn in tqdm(img_fns[:25]):
    result = reader.readtext(img_fn)
    img_id = img_fn.split('\\')[-1].split('.')[0]
    for bbox, text, conf in result:
        extracted_text.append({'img_id': img_id, 'text': text, 'confidence': conf})

# Convert extracted text into a DataFrame
extracted_text_df = pd.DataFrame(extracted_text)

# Display extracted text
print(extracted_text_df)


In [ ]:
import easyocr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

def plot_easyocr_results(img_fn, easyocr_df):
    img_id = img_fn.split('\\')[-1].split('.')[0]
    fig, ax = plt.subplots(figsize=(15, 10))

    image = plt.imread(img_fn)
    ax.imshow(image)

    # Filter EasyOCR results for the specific image
    easy_results = easyocr_df[easyocr_df['img_id'] == img_id]

    # Plot bounding boxes and text
    for index, row in easy_results.iterrows():
        bbox = row['bbox']
        text = row['text']
        confidence = row['conf']

        # Check if bbox is a list or numpy array
        if isinstance(bbox, np.ndarray):
            # Convert numpy array to list
            bbox = bbox.tolist()

        # Plot bounding box
        xmin, ymin = bbox[0][0], bbox[0][1]
        width, height = bbox[2][0] - bbox[0][0], bbox[2][1] - bbox[0][1]
        rect = plt.Rectangle((xmin, ymin), width, height,
                             fill=False, edgecolor='g', linewidth=2)
        ax.add_patch(rect)

        # Plot text and confidence score
        ax.text(xmin, ymin, f'{text} (Confidence: {confidence:.2f})',
                fontsize=12, bbox=dict(facecolor='g', alpha=0.5))

    ax.set_title('EasyOCR Results', fontsize=24)
    plt.show()

# List to store EasyOCR results for all images
easyocr_results = []

# Loop over images to extract text using EasyOCR
for img_fn in tqdm(img_fns[:25]):
    img_id = img_fn.split('\\')[-1].split('.')[0]
    result = reader.readtext(img_fn)

    # Append extracted text along with image ID to the results list
    for bbox, text, conf in result:
        easyocr_results.append({'img_id': img_id, 'bbox': bbox, 'text': text, 'conf': conf})

# Create a DataFrame from the EasyOCR results
easyocr_df = pd.DataFrame(easyocr_results)

# Loop over images and plot EasyOCR results
for img_fn in img_fns[:25]:
    plot_easyocr_results(img_fn, easyocr_df)
